<a href="https://colab.research.google.com/github/cgpilapanta/processamento/blob/main/script_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SCRIPT PARA BAIXAR DADOS PARA USO NO BERNESE**
---
*   Autor: Christian G. Pilapanta A.
*   Data : Outubro de 2024
*   Lotação: Departamento de Geomática - UFPR / Laboratorio de Geodésia Espacial e Hidrografia

---

## INI. Parametros gerais

### I.1 Importar bibliotecas necessárias

In [ ]:
# --------------------------------------------
# INSTALL PYTHON MODULES
# --------------------------------------------
!pip install gnsscal
!pip install unlzw3

import gnsscal, unlzw3

In [ ]:
# --------------------------------------------
# IMPORT PYTHON MODULES
# --------------------------------------------
import os, zipfile, shutil, subprocess, urllib.request, requests, pathlib, sys, gzip, zlib

from pathlib import Path
from datetime import date
from datetime import datetime
from google.colab import files
from ftplib import FTP_TLS, error_perm

### I.2. Definir parâmetros de usuário

In [ ]:
# --------------- SPECIFY USER INPUTS --------------- #
# --------------------------------------------
# RMBC STATIONS LIST (CAN BE MODIFIED)
# --------------------------------------------
sta_rbmc = ['apma', 'prma', 'onrj', 'mgbh']
sta_nigs = ['braz', 'brft', 'cuib', 'salu', 'savo',  'sptu', 'topl', 'ufpr']
# --------------------------------------------
# PERIOD OF OBSERVATION DATA (CAN BE MODIFIED)
# --------------------------------------------
INI_YY = 2021
END_YY = 2025
# --------------------------------------------
# MONTHS OF OBSERVATION DATA (CAN BE MODIFIED)
# --------------------------------------------
INT_MM = [1, 3, 5, 7, 9, 11]
# --------------------------------------------
# DAY OF OBSERVATION DATA (CAN BE MODIFIED)
# --------------------------------------------
INI_DD = 10
# --------------------------------------------
# HOURS OF VMF1 FUNCTIONS (CAN BE MODIFIED)
# --------------------------------------------
VMF_HH = [0, 6, 12, 18]
# --------------------------------------------
# E-MAIL FOR FTP SERVER
# --------------------------------------------
n_user = 'anonymous'
n_email = 'cgpilapanta@gmail.com'
# ---------------- END OF USER INPUTS --------------- #

## A. Baixar arquivos RINEX

### A.1. Rede IGS Internacional

In [ ]:
if not os.path.exists('/content/rinex/IGS'):
  os.makedirs('/content/rinex/IGS')

for year in range(INI_YY, END_YY):

  if not os.path.exists('/content/rinex/IGS/' + str(year)):
    os.makedirs('/content/rinex/IGS/' + str(year))

  if not os.path.exists('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'D'):
    os.makedirs('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'D')

  if not os.path.exists('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'G'):
    os.makedirs('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'G')

  if not os.path.exists('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'L'):
    os.makedirs('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'L')

  if not os.path.exists('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'M'):
    os.makedirs('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'M')

  if not os.path.exists('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'N'):
    os.makedirs('/content/rinex/IGS/' + str(year) + '/' + str(year)[2:] + 'N')

  for month in INT_MM:

    doy = datetime(year, month, INI_DD).timetuple().tm_yday

    for station in sta_nigs:

      file_count = '1'
      file_ext = '.zip'
      filename = station + str(doy).zfill(3) + file_count + file_ext

      url = 'https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/' + str(year) + '/' + str(doy).zfill(3) + '/' + filename

      request = requests.get(url)
      if request.status_code == 200:

          fullname = os.path.join('/content/rinex/IGS/' + str(year) + '/', filename)
          urllib.request.urlretrieve(url, fullname)

          with zipfile.ZipFile(fullname, 'r') as zip_ref:
            zip_ref.extractall('/content/rinex/IGS/' + str(year) + '/')

          os.remove(fullname)

          print('IGS/' + str(year) + '/' + filename + ' unzipped successfully')

      else:

          print('IGS/' + str(year) + '/' + filename + ' does not exist')

  folder = '/content/rinex/IGS/' + str(year)

  for path in pathlib.Path(folder).glob("*.*"):
    path2 = path.parent.joinpath(str(path.name).upper())
    shutil.move(path, path2)

  for path in pathlib.Path(folder).glob("*.*"):

    if str(path)[-1:] == "D":
      shutil.move(path, str(path)[:24] + str(path)[21:23] + 'D/' + str(path)[24:])
    elif str(path)[-1:] == "G":
      shutil.move(path, str(path)[:24] + str(path)[21:23] + 'G/' + str(path)[24:])
    elif str(path)[-1:] == "L":
      shutil.move(path, str(path)[:24] + str(path)[21:23] + 'L/' + str(path)[24:])
    elif str(path)[-1:] == "M":
      shutil.move(path, str(path)[:24] + str(path)[21:23] + 'M/' + str(path)[24:])
    elif str(path)[-1:] == "N":
      shutil.move(path, str(path)[:24] + str(path)[21:23] + 'N/' + str(path)[24:])

print('>>>>>>>>>> PROCESS COMPLETED <<<<<<<<<<')

### A.2. Rede Brasileira de Monitoramento Continuo. RBMC

In [ ]:
if not os.path.exists('/content/rinex/RBMC'):
  os.makedirs('/content/rinex/RBMC')

for year in range(INI_YY, END_YY):

  if not os.path.exists('/content/rinex/RBMC/' + str(year)):
    os.makedirs('/content/rinex/RBMC/' + str(year))

  if not os.path.exists('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'D'):
    os.makedirs('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'D')

  if not os.path.exists('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'G'):
    os.makedirs('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'G')

  if not os.path.exists('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'L'):
    os.makedirs('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'L')

  if not os.path.exists('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'M'):
    os.makedirs('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'M')

  if not os.path.exists('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'N'):
    os.makedirs('/content/rinex/RBMC/' + str(year) + '/' + str(year)[2:] + 'N')

  for month in INT_MM:

    doy = datetime(year, month, INI_DD).timetuple().tm_yday

    for station in sta_rbmc:

      file_count = '1'
      file_ext = '.zip'
      filename = station + str(doy).zfill(3) + file_count + file_ext

      url = 'https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/' + str(year) + '/' + str(doy).zfill(3) + '/' + filename

      request = requests.get(url)
      if request.status_code == 200:

          fullname = os.path.join('/content/rinex/RBMC/' + str(year) + '/', filename)
          urllib.request.urlretrieve(url, fullname)

          with zipfile.ZipFile(fullname, 'r') as zip_ref:
            zip_ref.extractall('/content/rinex/RBMC/' + str(year) + '/')

          os.remove(fullname)

          print('RBMC/' + str(year) + '/' + filename + ' unzipped successfully')

      else:

          print('RBMC/' + str(year) + '/' + filename + ' does not exist')

  folder = '/content/rinex/RBMC/' + str(year)

  for path in pathlib.Path(folder).glob("*.*"):
    path2 = path.parent.joinpath(str(path.name).upper())
    shutil.move(path, path2)

  for path in pathlib.Path(folder).glob("*.*"):

    if str(path)[-1:] == "D":
      shutil.move(path, str(path)[:25] + str(path)[22:24] + 'D/' + str(path)[25:])
    elif str(path)[-1:] == "G":
      shutil.move(path, str(path)[:25] + str(path)[22:24] + 'G/' + str(path)[25:])
    elif str(path)[-1:] == "L":
      shutil.move(path, str(path)[:25] + str(path)[22:24] + 'L/' + str(path)[25:])
    elif str(path)[-1:] == "M":
      shutil.move(path, str(path)[:25] + str(path)[22:24] + 'M/' + str(path)[25:])
    elif str(path)[-1:] == "N":
      shutil.move(path, str(path)[:25] + str(path)[22:24] + 'N/' + str(path)[25:])

print('>>>>>>>>>> PROCESS COMPLETED <<<<<<<<<<')

## B. Baixar arquivos de serviços

### B.1. Serviço IGS (SP3, CLK, ERP)

In [ ]:
def ftp_directory_exist(ftp, directory_path):
  try:
    ftp.cwd(directory_path)
    return True
  except error_perm as e:
    if "550" in str(e):
      return False
    else:
      raise

if not os.path.exists('/content/products/IGS'):
  os.makedirs('/content/products/IGS')

for year in range(INI_YY, END_YY):

  if not os.path.exists('/content/products/IGS/' + str(year)):
    os.makedirs('/content/products/IGS/' + str(year))

  for month in INT_MM:

    # Day of year
    doy = datetime(year, month, INI_DD).timetuple().tm_yday
    # GPS Week
    week = gnsscal.date2gpswd(date(year, month, INI_DD))[0]
    # Day of GPS week
    dowk = gnsscal.date2gpswd(date(year, month, INI_DD))[1]
    # First day of week as date
    dates = gnsscal.gpswd2date(week, 0)
    # First day of week as DOY
    idoy = dates.timetuple().tm_yday

    directory = 'gps/products/' + str(week)

    ftps = FTP_TLS(host = 'gdc.cddis.eosdis.nasa.gov')
    ftps.login(user = n_user, passwd= n_email)
    ftps.prot_p()

    if week < 2238:
      file_sp3 = 'igs' + str(week) + str(dowk) + '.sp3.Z'
      file_clk = 'igs' + str(week) + str(dowk) + '.clk.Z'
      file_erp = 'igs' + str(week) + '7.erp.Z'

      if ftp_directory_exist(ftps, directory):
        ftps.retrbinary("RETR " + file_sp3, open('/content/products/IGS/' + str(year) + '/' + file_sp3, 'wb').write)

        uncompressed_data = unlzw3.unlzw(Path('/content/products/IGS/' + str(year) + '/' + file_sp3).read_bytes())
        data = uncompressed_data.decode('utf-8')

        with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + str(dowk) + '.PRE', 'w') as f:
          f.writelines(data)

        print(str(year) + '/' + file_sp3 + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_sp3)

        ftps.retrbinary("RETR " + file_clk, open('/content/products/IGS/' + str(year) + '/' + file_clk, 'wb').write)

        uncompressed_data = unlzw3.unlzw(Path('/content/products/IGS/' + str(year) + '/' + file_clk).read_bytes())
        data = uncompressed_data.decode('utf-8')

        with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + str(dowk) + '.CLK', 'w') as f:
          f.writelines(data)

        print(str(year) + '/' + file_clk + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_clk)

        ftps.retrbinary("RETR " + file_erp, open('/content/products/IGS/' + str(year) + '/' + file_erp, 'wb').write)

        uncompressed_data = unlzw3.unlzw(Path('/content/products/IGS/' + str(year) + '/' + file_erp).read_bytes())
        data = uncompressed_data.decode('utf-8')

        with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + '7.IEP', 'w') as f:
          f.writelines(data)

        print(str(year) + '/' + file_erp + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_erp)

    elif week >= 2238:
      file_nat = 'IGS0OPSFIN_'
      file_ext_sp3 = '0000_01D_15M_ORB.SP3.gz'
      file_ext_clk = '0000_01D_05M_CLK.CLK.gz'
      file_ext_erp = '0000_07D_01D_ERP.ERP.gz'

      file_sp3 = file_nat + str(year) + str(doy).zfill(3) + file_ext_sp3
      file_clk = file_nat + str(year) + str(doy).zfill(3) + file_ext_clk
      file_erp = file_nat + str(year) + str(idoy).zfill(3) + file_ext_erp

      if ftp_directory_exist(ftps, directory):
        ftps.retrbinary("RETR " + file_sp3, open('/content/products/IGS/' + str(year) + '/' + file_sp3, 'wb').write)

        with gzip.open('/content/products/IGS/' + str(year) + '/' + file_sp3, 'rb') as f_in:
          with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + str(dowk) + '.PRE', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(str(year) + '/' + file_sp3 + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_sp3)

        ftps.retrbinary("RETR " + file_clk, open('/content/products/IGS/' + str(year) + '/' + file_clk, 'wb').write)

        with gzip.open('/content/products/IGS/' + str(year) + '/' + file_clk, 'rb') as f_in:
          with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + str(dowk) + '.CLK', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(str(year) + '/' + file_clk + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_clk)

        ftps.retrbinary("RETR " + file_erp, open('/content/products/IGS/' + str(year) + '/' + file_erp, 'wb').write)

        with gzip.open('/content/products/IGS/' + str(year) + '/' + file_erp, 'rb') as f_in:
          with open('/content/products/IGS/' + str(year) + '/igs' + str(week) + '7.IEP', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(str(year) + '/' + file_erp + ' unzipped successfully')
        os.remove('/content/products/IGS/' + str(year) + '/' + file_erp)

      else:
        print('Directory does not exist')

  folder = '/content/products/IGS/' + str(year)

  for path in pathlib.Path(folder).glob("*.*"):
      path2 = path.parent.joinpath(str(path.name).upper())
      shutil.move(path, path2)

print('>>>>>>>>>> PROCESS COMPLETED <<<<<<<<<<')

### B.2. Serviço CODE (IONEX)

In [ ]:
if not os.path.exists('/content/products/CODE'):
  os.makedirs('/content/products/CODE')

for year in range(INI_YY, END_YY):

  if not os.path.exists('/content/products/CODE/' + str(year)):
    os.makedirs('/content/products/CODE/' + str(year))

  for month in INT_MM:

    # Day of year
    doy = datetime(year, month, INI_DD).timetuple().tm_yday
    # GPS Week
    week = gnsscal.date2gpswd(date(year, month, INI_DD))[0]
    # Day of GPS week
    dowk = gnsscal.date2gpswd(date(year, month, INI_DD))[1]

    if week < 2238:
      filename = 'COD' + str(week) + str(dowk) + '.ION.Z'

      url = 'http://ftp.aiub.unibe.ch/CODE/' + str(year) + '/' + filename
      request = requests.get(url)

      if request.status_code == 200:

        fullname = os.path.join('/content/products/CODE/' + str(year) + '/', filename)
        urllib.request.urlretrieve(url, fullname)

        uncompressed_data = unlzw3.unlzw(Path('/content/products/CODE/' + str(year) + '/' + filename).read_bytes())
        data = uncompressed_data.decode('utf-8')

        with open('/content/products/CODE/' + str(year) + '/COD' + str(week) + str(dowk) + '.ION', 'w') as f:
          f.writelines(data)

        print(str(year) + '/' + filename + ' unzipped successfully')
        os.remove('/content/products/CODE/' + str(year) + '/' + filename)

    elif week >= 2238:
      file_nat = 'COD0OPSFIN_'
      file_ext = '0000_01D_01H_GIM.ION.gz'

      filename = file_nat + str(year) + str(doy).zfill(3) + file_ext

      url = 'http://ftp.aiub.unibe.ch/CODE/' + str(year) + '/' + filename
      request = requests.get(url)

      if request.status_code == 200:

        fullname = os.path.join('/content/products/CODE/' + str(year) + '/', filename)
        urllib.request.urlretrieve(url, fullname)

        with gzip.open('/content/products/CODE/' + str(year) + '/' + filename, 'rb') as f_in:
          with open('/content/products/CODE/' + str(year) + '/COD' + str(week) + str(dowk) + '.ION', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

        print(str(year) + '/' + filename + ' unzipped successfully')
        os.remove('/content/products/CODE/' + str(year) + '/' + filename)

print('>>>>>>>>>> PROCESS COMPLETED <<<<<<<<<<')

### Serviço TU Wien (VMF1 Grids)

In [ ]:
if not os.path.exists('/content/products/WIEN'):
  os.makedirs('/content/products/WIEN')

for year in range(INI_YY, END_YY):

  if not os.path.exists('/content/products/WIEN/' + str(year)):
    os.makedirs('/content/products/WIEN/' + str(year))

  for month in INT_MM:

    # Day of year
    doy = datetime(year, month, INI_DD).timetuple().tm_yday
    # GPS Week
    week = gnsscal.date2gpswd(date(year, month, INI_DD))[0]
    # Day of GPS week
    dowk = gnsscal.date2gpswd(date(year, month, INI_DD))[1]

    file_nat = 'VMFG_'

    files_list = []

    for hours in VMF_HH:

      file_ext = '.H' + str(hours).zfill(2)
      filename = 'VMFG_' + str(year) + str(month).zfill(2) + str(INI_DD).zfill(2) + file_ext

      url = 'https://vmf.geo.tuwien.ac.at/trop_products/GRID/2.5x2/VMF1/VMF1_OP/' + str(year) + '/' + filename

      request = requests.get(url)
      if request.status_code == 200:

        fullname = os.path.join('/content/products/WIEN/' + str(year) + '/', filename)
        urllib.request.urlretrieve(url, fullname)

        files_list.append(fullname)

    # 00 HOURS OF NEXT DAY
    filename = 'VMFG_' + str(year) + str(month).zfill(2) + str(INI_DD+1).zfill(2) + '.H00'

    url = 'https://vmf.geo.tuwien.ac.at/trop_products/GRID/2.5x2/VMF1/VMF1_OP/' + str(year) + '/' + filename

    request = requests.get(url)
    if request.status_code == 200:

      fullname = os.path.join('/content/products/WIEN/' + str(year) + '/', filename)
      urllib.request.urlretrieve(url, fullname)

      files_list.append(fullname)

    with open('/content/products/WIEN/' + str(year) + '/VMF' + str(year)[2:] + str(doy) + '.GRD', 'w') as outfile:
      for fname in files_list:
        with open(fname) as infile:
          for line in infile:
            outfile.write(line)

    print('/content/products/WIEN/' + str(year) + '/VMF' + str(year)[2:] + str(doy).zfill(3) + '.GRD' + ' was successfully created')

    dir_VMF = os.listdir('/content/products/WIEN/' + str(year))

    for item in files_list:
      os.remove(item)

## FIN. Zipar e baixar os dados

In [ ]:
!zip -r rinex.zip "rinex"
!zip -r products.zip "products"

files.download('rinex.zip')
files.download('products')

#shutil.rmtree('/content/rinex', ignore_errors=True)
#shutil.rmtree('/content/products', ignore_errors=True)